In [1]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import os
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error

from src.dynamic_pricing_data_loader import cargar_y_preparar_datos

In [2]:
def Get_Data():
    # Obtener el directorio de trabajo actual (ruta principal del proyecto).
    ruta_principal = os.getcwd()

    # Construir la ruta al archivo de configuración "config/config.json".
    config_path = os.path.join(ruta_principal, "config", "config.json")

    # Llamar a la función externa que carga y realiza preprocesamiento inicial.
    Frame = cargar_y_preparar_datos(config_path, ruta_principal)

    # Seleccionar solo las columnas relevantes para el análisis.
    Frame = Frame[["FECHA_CORRIDA", "HORA_SALIDA_CORRIDA", "CLASE_SERVICIO", 'IVA_TARIFA_BASE_TRAMO',
    "PAX_SUBEN", "TARIFA_BASE_TRAMO",'FECHA_OPERACION', 'HORA_OPERACION','VENTA','DISPONIBILIDAD_TRAMO',
    'HORAS_ANTICIPACION','ORIGEN', 'DESTINO','TIPO_CLIENTE','NUM_ASIENTO'
                  ]].copy()

    return Frame

In [3]:
def Prepare_Data(df):
    #df=Get_Data()
    # Se filtra el DataFrame para incluir solo ventas mayores que cero.
    df = df[df['VENTA'] > 0]
    df['FECHA_OPERACION'] = pd.to_datetime(df['FECHA_OPERACION'])
    fecha_maxima = df['FECHA_OPERACION'].max()
    df = df[df['FECHA_OPERACION'] < fecha_maxima].copy()
    df['FECHA_CORRIDA'] = pd.to_datetime(df['FECHA_CORRIDA'])
    
    df["HORA_SALIDA_CORRIDA"] = pd.to_datetime(df["HORA_SALIDA_CORRIDA"])
    
    df['TBT']= df['TARIFA_BASE_TRAMO']-df['IVA_TARIFA_BASE_TRAMO']
    df['%_dif_TBT_Venta']= (df['TBT']-df['VENTA'])/df['TBT']
    df['TIPO_CLASE'] = np.where(
        df['CLASE_SERVICIO'].astype(str).str.contains('DOS PISOS', case=False, na=False),
        'DOS',
        'UNO'
    )
    return df 

In [4]:
def Data4RedNeuronal(df_1):
    df=Prepare_Data(df_1)
    df_total= pd.DataFrame()
    df_total['Origen-Destino'] = df['ORIGEN'].astype(str) + '-' + df['DESTINO'].astype(str)
    df_total['DiaSemana_Corrida']=df['FECHA_CORRIDA'].dt.dayofweek
    df_total['Hora_Corrida']=df['HORA_SALIDA_CORRIDA'].dt.hour
    df_total[['NUM_ASIENTO','HORAS_ANTICIPACION','%_dif_TBT_Venta']]=df[['NUM_ASIENTO','HORAS_ANTICIPACION','%_dif_TBT_Venta']].copy()
    df_total['Mes_Corrida']=df['FECHA_CORRIDA'].dt.month
    df_total['Año_Corrida']=df['FECHA_CORRIDA'].dt.year
    df_total['Buen_Dia'] = df['FECHA_CORRIDA'].dt.dayofweek.isin([4,5,6,0]).astype(int)
    df_total['Buena_Hora'] = df['HORA_SALIDA_CORRIDA'].dt.hour.isin([23,17,18,19,20]).astype(int)
    df_total['Buen_Mes'] = df['FECHA_CORRIDA'].dt.month.isin([3,4,5,6]).astype(int)
    df_total['Buen_Asiento'] = df['NUM_ASIENTO'].isin([1,2,3,4,5,6,7,8,9,10]).astype(int)
    # Crea un nuevo DataFrame con las variables dummy (codificación one-hot)
    df_dummies = pd.get_dummies(
        df['TIPO_CLIENTE'],
        prefix='TIPO_CLIENTE', # Prefijo para las nuevas columnas (ej: TIPO_CLIENTE_A)
        drop_first=False        # Elimina la primera categoría para evitar multicolinealidad
    ).astype(int)
    
    # Crea un nuevo DataFrame con las variables dummy (codificación one-hot)
    df_dummies1 = pd.get_dummies(
        df['TIPO_CLASE'],
        prefix='PISO', 
        drop_first=False
    ).astype(int)
    
    # Une las nuevas columnas dummy al DataFrame original
    df_total = pd.concat([df_total, df_dummies,df_dummies1], axis=1)
    df_total['VENTA']=df['VENTA'].copy()

    return df_total

In [5]:
Frame=Get_Data()
FrameN=Data4RedNeuronal(Frame.copy())

Memoria usada antes: 623.00 MB
Memoria usada después: 121.10 MB
Reducción: 80.6%


In [6]:
#FrameN.to_excel('PruebaData4RedN.xlsx')

In [7]:
def GetFlag(datos):

    asimetria_pandas = datos.skew()
    print(f"Coeficiente de Asimetría: {asimetria_pandas:.4f}")
    
    if asimetria_pandas > 1.0:
        print("La asimetría es alta (> 1.0). La transformación logarítmica es altamente recomendable.")
        Bandera=True
    elif asimetria_pandas > 0.5:
        print("La asimetría es moderada (> 0.5). La transformación logarítmica podría ser beneficiosa.")
        Bandera=True
    else:
        print("La asimetría es baja. Una transformación no es necesaria.")
        Bandera=False
        
    return Bandera

In [8]:
"""datos=FrameN['VENTA']
plt.hist(datos, bins=10, edgecolor='black')

# 3. Añadir etiquetas y título
plt.title('Histograma Básico con Matplotlib')
plt.xlabel('Valor')
plt.ylabel('Frecuencia')

# 4. Mostrar el gráfico
plt.show()"""

"datos=FrameN['VENTA']\nplt.hist(datos, bins=10, edgecolor='black')\n\n# 3. Añadir etiquetas y título\nplt.title('Histograma Básico con Matplotlib')\nplt.xlabel('Valor')\nplt.ylabel('Frecuencia')\n\n# 4. Mostrar el gráfico\nplt.show()"

In [9]:
"""plt.hist(Y_log, bins=10, edgecolor='black')

# 3. Añadir etiquetas y título
plt.title('Histograma Básico con Matplotlib')
plt.xlabel('Valor')
plt.ylabel('Frecuencia')

# 4. Mostrar el gráfico
plt.show()"""

"plt.hist(Y_log, bins=10, edgecolor='black')\n\n# 3. Añadir etiquetas y título\nplt.title('Histograma Básico con Matplotlib')\nplt.xlabel('Valor')\nplt.ylabel('Frecuencia')\n\n# 4. Mostrar el gráfico\nplt.show()"

In [10]:
def GetTrainingForm(df,Bandera):
    # Definir la variable objetivo (Y)
    Y = df['VENTA']
    
    if Bandera:
        # Aplicar la transformación logarítmica a Y
        Y_log = np.log(Y)
    else:
        Y_log = Y.copy()
    
    # Eliminar la variable VENTA del dataframe de features (X)
    X = df.drop('VENTA', axis=1) 
    
    categorical_features= 'Origen-Destino'
    df_ohe = pd.get_dummies(X[categorical_features]).astype(int)
    
    # Columnas numéricas que necesitan Estandarización
    # Excluimos las binarias/dummies que ya están bien escaladas (0 o 1)
    numeric_features = [
        'DiaSemana_Corrida', 'Hora_Corrida', 'NUM_ASIENTO', 
        'HORAS_ANTICIPACION', '%_dif_TBT_Venta', 'Mes_Corrida','Año_Corrida'
    ]
    
    # Columnas binarias (se dejan pasar sin transformación)
    binary_features = [col for col in X.columns if col not in [categorical_features] + numeric_features]
    
    indice_correcto = X[numeric_features].index # o df_ohe.index
    
    scaler = StandardScaler()
    # 2. Convierte el array escalado (NumPy) a DataFrame, ASIGNANDO el índice correcto
    X_escalado_array = scaler.fit_transform(X[numeric_features])
    X_escalado = pd.DataFrame(X_escalado_array, 
                              index=indice_correcto, # <-- ¡CLAVE!
                              columns=numeric_features)
    
    X_processed= pd.concat([df_ohe, X_escalado,X[binary_features]], axis=1)

    return X_processed, Y_log

In [11]:
def TrainingNet(X_processed,Y_log):
    # División del 80% para entrenamiento y 20% para prueba
    X_train, X_test, Y_train_log, Y_test_log = train_test_split(
        X_processed, 
        Y_log, 
        test_size=0.2, 
        random_state=42 # Para asegurar resultados reproducibles
    )
    
    input_feature_count = X_train.shape[1] 
    
    print(f"\n--- Resumen de la División ---")
    print(f"Número total de features (columnas) para la red neuronal: {input_feature_count}")
    print(f"Tamaño de X_train (Entrenamiento): {X_train.shape}")
    print(f"Tamaño de X_test (Prueba): {X_test.shape}")
    print(f"Tamaño de Y_train_log (Objetivo de Entrenamiento): {Y_train_log.shape}")
    
    # --- 1. Definir el número de features de entrada ---
    # (Esto debe ser el número de columnas de tu X_train después del OHE y estandarización)
    input_shape = X_train.shape[1] 
    
    # --- 2. CONSTRUCCIÓN DEL MODELO ---
    model = Sequential([
        # Capa Oculta 1
        Dense(128, activation='relu', input_shape=(input_shape,)),
        
        # Capa Oculta 2 (Regularización para evitar overfitting)
        # Aquí puedes añadir 'Dropout' si notas que el modelo se sobreajusta
        Dense(64, activation='relu'), 
        
        # Capa de Salida: 1 neurona y activación lineal para regresión
        Dense(1, activation='linear') 
    ])
    
    # --- 3. COMPILACIÓN DEL MODELO ---
    model.compile(
        optimizer='adam',
        loss='mse',           # Función de pérdida: Error Cuadrático Medio
        metrics=['mae', 'mse']  # Métricas a monitorear: MAE y MSE
    )
    
    # --- 4. ENTRENAMIENTO (Ejemplo) ---
    history = model.fit(
        X_train, 
        Y_train_log,  # ¡Usamos la variable VENTA transformada con logaritmo!
        epochs=10, 
        batch_size=32, 
        validation_split=0.2, # Usamos el 20% para validación interna
        verbose=1
    )
    
    PredictingNet(model,X_test, Y_test_log)
    return model

In [12]:
def PredictingNet(model,X_test, Y_test_log):
    # 'model' es tu red neuronal entrenada
    # 'X_test' son tus features de prueba (escalados y codificados)
    Y_pred_log = model.predict(X_test)
    
    if Bandera:
        # Revertir la predicción logarítmica a la escala de precio real
        Y_pred_real = np.exp(Y_pred_log)
        # Revertir los valores reales de prueba (Y_test_log) a la escala de precio real
        # Esto es para compararlos directamente
        Y_test_real = np.exp(Y_test_log) 
    else:
        Y_pred_real= Y_pred_log.copy()
        Y_test_real = Y_test_log.copy()
    
    
    # Calcular el MAE real
    mae_real = mean_absolute_error(Y_test_real, Y_pred_real)
    
    print(f"\nEl Error Absoluto Medio (MAE) final es de: {mae_real:,.2f} [Moneda]")
    
    #  Calcular el Error Cuadrático Medio (MSE)
    mse_real = mean_squared_error(Y_test_real, Y_pred_real)
    
    # Calcular la Raíz del Error Cuadrático Medio (RMSE)
    #    La RMSE es simplemente la raíz cuadrada del MSE
    rmse_real = np.sqrt(mse_real)
    
    print(f"\nLa Raíz del Error Cuadrático Medio (RMSE) final es de: {rmse_real:,.2f} [Moneda]")

In [13]:
Bandera = GetFlag(FrameN['VENTA'])
X_processed, Y_log = GetTrainingForm(FrameN.copy(),Bandera)

Coeficiente de Asimetría: -0.5957
La asimetría es baja. Una transformación no es necesaria.


In [14]:
model= TrainingNet(X_processed,Y_log)
print(model.summary())


--- Resumen de la División ---
Número total de features (columnas) para la red neuronal: 20
Tamaño de X_train (Entrenamiento): (284547, 20)
Tamaño de X_test (Prueba): (71137, 20)
Tamaño de Y_train_log (Objetivo de Entrenamiento): (284547,)
Epoch 1/10
7114/7114 ━━━━━━━━━━━━━━━━━━━━ 11s 1ms/step - loss: 18300.2773 - mae: 44.3896 - mse: 18300.2773 - val_loss: 132.5484 - val_mae: 7.4194 - val_mse: 132.5484
Epoch 2/10
7114/7114 ━━━━━━━━━━━━━━━━━━━━ 10s 1ms/step - loss: 120.6292 - mae: 7.5019 - mse: 120.6292 - val_loss: 103.6655 - val_mae: 6.6937 - val_mse: 103.6655
Epoch 3/10
7114/7114 ━━━━━━━━━━━━━━━━━━━━ 10s 1ms/step - loss: 106.6296 - mae: 6.9037 - mse: 106.6296 - val_loss: 97.9757 - val_mae: 6.3931 - val_mse: 97.9757
Epoch 4/10
7114/7114 ━━━━━━━━━━━━━━━━━━━━ 9s 1ms/step - loss: 102.1416 - mae: 6.7044 - mse: 102.1416 - val_loss: 94.6241 - val_mae: 6.3055 - val_mse: 94.6241
Epoch 5/10
7114/7114 ━━━━━━━━━━━━━━━━━━━━ 10s 1ms/step - loss: 99.5385 - mae: 6.6183 - mse: 99.5385 - val_loss: 94.

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ dense (Dense)                        │ (None, 128)                 │           2,688 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 64)                  │           8,256 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_2 (Dense)                      │ (None, 1)                   │              65 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 33,029 (129.02 KB)

 Trainable params: 11,009 (43.00 KB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 22,020 (86.02 KB)

None


In [15]:
def DataForecasting(df,FrameN):

    df['TIPO_CLASE'] = np.where(
        df['CLASE_SERVICIO'].astype(str).str.contains('DOS PISOS', case=False, na=False),
        'DOS',
        'UNO'
    )
    df["HORA_SALIDA_CORRIDA"] = pd.to_datetime(df["HORA_SALIDA_CORRIDA"])
    df['FECHA_CORRIDA'] = pd.to_datetime(df['FECHA_CORRIDA'])
    df_total = pd.DataFrame(columns=FrameN.columns)
    print(df_total.shape)
    
    df_total['Origen-Destino'] = df['ORIGEN'].astype(str) + '-' + df['DESTINO'].astype(str)
    df_total['DiaSemana_Corrida']=df['FECHA_CORRIDA'].dt.dayofweek
    df_total['Hora_Corrida']=df['HORA_SALIDA_CORRIDA'].dt.hour
    df_total[['NUM_ASIENTO','HORAS_ANTICIPACION']]=df[['NUM_ASIENTO','HORAS_ANTICIPACION']].copy()
    df_total['%_dif_TBT_Venta']=FrameN['%_dif_TBT_Venta'].mean()
    df_total['Mes_Corrida']=df['FECHA_CORRIDA'].dt.month
    df_total['Año_Corrida']=df['FECHA_CORRIDA'].dt.year
    df_total['Buen_Dia'] = df['FECHA_CORRIDA'].dt.dayofweek.isin([4,5,6,0]).astype(int)
    df_total['Buena_Hora'] = df['HORA_SALIDA_CORRIDA'].dt.hour.isin([23,17,18,19,20]).astype(int)
    df_total['Buen_Mes'] = df['FECHA_CORRIDA'].dt.month.isin([3,4,5,6]).astype(int)
    df_total['Buen_Asiento'] = df['NUM_ASIENTO'].isin([1,2,3,4,5,6,7,8,9,10]).astype(int)
    # Crea un nuevo DataFrame con las variables dummy (codificación one-hot)
    df_dummies = pd.get_dummies(
        df['TIPO_CLIENTE'],
        prefix='TIPO_CLIENTE', # Prefijo para las nuevas columnas (ej: TIPO_CLIENTE_A)
        drop_first=False        # Elimina la primera categoría para evitar multicolinealidad
    ).astype(int)

    df_total[df_dummies.columns]= df_dummies[df_dummies.columns].copy()
    # Crea un nuevo DataFrame con las variables dummy (codificación one-hot)
    df_dummies1 = pd.get_dummies(
        df['TIPO_CLASE'],
        prefix='PISO', 
        drop_first=False
    ).astype(int)

    df_total[df_dummies1.columns]= df_dummies1[df_dummies1.columns].copy()
    # Une las nuevas columnas dummy al DataFrame original

    df_total['VENTA']=df['VENTA'].copy()

    #df_total=df_total.fillna(0)
    
    return df_total

In [16]:
def PrepareData4Fore(df):
    #df=Get_Data()
    # Se filtra el DataFrame para incluir solo ventas mayores que cero.
    df = df[df['VENTA'] > 0]
    df['FECHA_OPERACION'] = pd.to_datetime(df['FECHA_OPERACION'])
    fecha_maxima = df['FECHA_OPERACION'].max()
    df = df[df['FECHA_OPERACION'] == fecha_maxima].copy()
    return df 

In [17]:
def GetPredictingForm(Fore,cols):
    X1 = Fore.drop('VENTA', axis=1)
    
    X_final=pd.DataFrame(columns=cols)
    
    categorical_features= 'Origen-Destino'
    df_ohe = pd.get_dummies(X1[categorical_features]).astype(int)
    
    # Columnas numéricas que necesitan Estandarización
    # Excluimos las binarias/dummies que ya están bien escaladas (0 o 1)
    numeric_features = [
        'DiaSemana_Corrida', 'Hora_Corrida', 'NUM_ASIENTO', 
        'HORAS_ANTICIPACION', '%_dif_TBT_Venta', 'Mes_Corrida','Año_Corrida'
    ]
    
    # Columnas binarias (se dejan pasar sin transformación)
    binary_features = [col for col in X1.columns if col not in [categorical_features] + numeric_features]
    
    indice_correcto = X1[numeric_features].index # o df_ohe.index
    
    scaler = StandardScaler()
    # 2. Convierte el array escalado (NumPy) a DataFrame, ASIGNANDO el índice correcto
    X_escalado_array = scaler.fit_transform(X1[numeric_features])
    X_escalado = pd.DataFrame(X_escalado_array, 
                              index=indice_correcto, # <-- ¡CLAVE!
                              columns=numeric_features)
    
    X_processed1= pd.concat([df_ohe, X_escalado,X1[binary_features]], axis=1)
    
    X_final[X_processed1.columns]= X_processed1[X_processed1.columns].copy()
    X_final=X_final.fillna(0)

    return X_final

In [18]:
def GetValues(model,Fore,X_final):
    # 'model' es tu red neuronal entrenada
    # 'X_test' son tus features de prueba (escalados y codificados)
    Y_pred_log = model.predict(X_final)
    
    Y_R_real = Fore['VENTA']
                
    if Bandera:
        # Revertir la predicción logarítmica a la escala de precio real
        Y_pred_real = np.exp(Y_pred_log)
    else:
        Y_pred_real= Y_pred_log.copy()
    
    # Calcular el MAE real
    mae_real = mean_absolute_error(Y_R_real, Y_pred_real)
    
    print(f"\nEl Error Absoluto Medio (MAE) final es de: {mae_real:,.2f} [Moneda]")
    return Y_pred_real

In [19]:
df_today= PrepareData4Fore(Frame.copy())
Fore=DataForecasting(df_today,FrameN)
X_final= GetPredictingForm(Fore, X_processed.columns)

(0, 20)


In [20]:
GetValues(model,Fore,X_final)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step

El Error Absoluto Medio (MAE) final es de: 238.25 [Moneda]


array([[949.10065],
       [949.549  ],
       [949.41455],
       [949.63873],
       [950.04236],
       [947.7451 ],
       [947.476  ],
       [948.9037 ],
       [949.3522 ],
       [949.4419 ],
       [949.397  ],
       [948.87415],
       [947.51685],
       [949.3207 ],
       [949.2113 ],
       [948.81   ],
       [948.3141 ],
       [948.3589 ],
       [955.2208 ],
       [949.38293],
       [950.14526],
       [948.8274 ],
       [948.8722 ],
       [948.7825 ],
       [948.7803 ],
       [949.4103 ],
       [949.23083],
       [949.3206 ],
       [949.3655 ],
       [949.45514],
       [949.5    ]], dtype=float32)